# Appwrite Fact Check Notebook

Use this notebook to verify dashboard queries directly against Appwrite Databases.

- Requires environment variables: `APPWRITE_ENDPOINT`, `APPWRITE_PROJECT_ID`, `APPWRITE_API_KEY` (server key), optional `NEXT_PUBLIC_APPWRITE_DATABASE_ID`.
- Collections used: `transactions_dev`, `balances_dev` per the project `specs.md`.
- Set the date range below and run the cells to fetch raw rows and computed metrics.



In [13]:
# Parameters
from datetime import date, timedelta

# Set your date range here
DATE_FROM = date.today() - timedelta(days=30)
DATE_TO = date.today()

# Format to YYYY-MM-DD (matches dashboard filters)
FROM_STR = DATE_FROM.strftime("%Y-%m-%d")
TO_STR = DATE_TO.strftime("%Y-%m-%d")

FROM_STR, TO_STR


('2025-08-27', '2025-09-26')

In [14]:
# Appwrite client setup
import os
from appwrite.client import Client
from appwrite.services.databases import Databases

APPWRITE_ENDPOINT = os.getenv("APPWRITE_ENDPOINT") or os.getenv("NEXT_PUBLIC_APPWRITE_ENDPOINT")
APPWRITE_PROJECT_ID = os.getenv("APPWRITE_PROJECT_ID") or os.getenv("NEXT_PUBLIC_APPWRITE_PROJECT_ID")
APPWRITE_API_KEY = os.getenv("APPWRITE_API_KEY")  # Server API key
DATABASE_ID = os.getenv("NEXT_PUBLIC_APPWRITE_DATABASE_ID", "68d42ac20031b27284c9")
TRANSACTIONS_COLLECTION_ID = os.getenv("APPWRITE_TRANSACTIONS_COLLECTION_ID", "transactions_dev")
BALANCES_COLLECTION_ID = os.getenv("APPWRITE_BALANCES_COLLECTION_ID", "balances_dev")

assert APPWRITE_ENDPOINT and APPWRITE_PROJECT_ID and APPWRITE_API_KEY, "Missing required Appwrite env vars"

client = Client()
client.set_endpoint(APPWRITE_ENDPOINT)
client.set_project(APPWRITE_PROJECT_ID)
client.set_key(APPWRITE_API_KEY)

db = Databases(client)

#APPWRITE_ENDPOINT, APPWRITE_PROJECT_ID, DATABASE_ID


In [15]:
# Helpers
from appwrite.query import Query
from typing import List, Dict, Any

def list_all(db: Databases, database_id: str, collection_id: str, queries: List[Any]) -> List[Dict[str, Any]]:
    # Appwrite returns paginated results; fetch in batches
    out: List[Dict[str, Any]] = []
    cursor = None
    while True:
        q = list(queries)
        q.append(Query.limit(100))
        if cursor:
            q.append(Query.cursor_after(cursor))
        res = db.list_documents(database_id, collection_id, q)
        docs = res["documents"]
        out.extend(docs)
        if len(docs) < 100:
            break
        cursor = docs[-1]["$id"]
    return out

# Date range filters (bookingDate is stored as date string)
base_tx_filters = [
    Query.greater_than_equal("bookingDate", FROM_STR),
    Query.less_than_equal("bookingDate", TO_STR),
]
base_user_filters = []  # optionally add Query.equal("userId", "<your-user-id>") for single user



In [16]:
# Fetch transactions (raw)
queries = base_user_filters + base_tx_filters
transactions = list_all(db, DATABASE_ID, TRANSACTIONS_COLLECTION_ID, queries)
len(transactions), transactions[:2]


(46,
 [{'userId': '68d446e7bf3ed043310a',
   'accountId': '4972c2c4-65b8-4fb2-b97a-8bc7fca6d2c2',
   'transactionId': '2025092401645717-1',
   'amount': '-95.35',
   'currency': 'EUR',
   'bookingDate': '2025-09-24',
   'bookingDateTime': None,
   'valueDate': '2025-09-24',
   'description': 'Freshto Ideal Clerkenwell',
   'counterparty': 'Freshto Ideal',
   'raw': '{"transactionId":"2025092401645717-1","entryReference":"2025092401645717-1","bookingDate":"2025-09-24","valueDate":"2025-09-24","transactionAmount":{"amount":"-95.35","currency":"EUR"},"creditorName":"Freshto Ideal","remittanceInformationUnstructured":"Freshto Ideal Clerkenwell","bankTransactionCode":"PMNT","proprietaryBankTransactionCode":"PURCHASE","internalTransactionId":"fe79bfedd5972a526386b59918457d22"}',
   '$id': '68d5c0030031e24894b2',
   '$sequence': 353,
   '$createdAt': '2025-09-25T22:19:47.852+00:00',
   '$updatedAt': '2025-09-25T22:19:47.852+00:00',
   '$permissions': [],
   '$databaseId': '68d42ac20031b27284c

In [17]:
# Compute income / expenses
income = 0.0
expenses = 0.0
for t in transactions:
    amt = float(t.get("amount", 0))
    if amt > 0:
        income += amt
    else:
        expenses += abs(amt)
net = income - expenses
saving_rate = ((income - expenses) / income * 100) if income > 0 else 0.0
{
    "income": round(income, 2),
    "expenses": round(expenses, 2),
    "net": round(net, 2),
    "savingRatePct": round(saving_rate, 2)
}


{'income': 7447.1, 'expenses': 6946.8, 'net': 500.3, 'savingRatePct': 6.72}

In [18]:
# Category breakdown (expenses only)
from collections import defaultdict

by_cat = defaultdict(float)
for t in transactions:
    amt = float(t.get("amount", 0))
    if amt < 0:
        cat = t.get("category") or "Uncategorized"
        by_cat[cat] += abs(amt)

# Normalize to percent
total_exp = sum(by_cat.values()) or 1.0
breakdown = [
    {"name": k, "amount": round(v, 2), "percent": round(v/total_exp*100, 2)}
    for k, v in sorted(by_cat.items(), key=lambda kv: kv[1], reverse=True)
]

breakdown[:10]


[{'name': 'Uncategorized', 'amount': 6946.8, 'percent': 100.0}]

In [19]:
# Balance snapshot (interimAvailable, latest per account)
from appwrite.query import Query

bal_docs = db.list_documents(
    DATABASE_ID,
    BALANCES_COLLECTION_ID,
    [
        Query.equal("balanceType", "interimAvailable"),
        Query.order_desc("referenceDate"),
        Query.limit(1000)
    ]
)["documents"]

seen = set()
total_balance = 0.0
for b in bal_docs:
    acct = b.get("accountId")
    if acct and acct not in seen:
        seen.add(acct)
        total_balance += float(b.get("balanceAmount", 0))

total_balance


3826.24

# Pandas

In [20]:
import pandas as pd
df = pd.DataFrame(transactions)
df

,userId,accountId,transactionId,amount,currency,bookingDate,bookingDateTime,valueDate,description,counterparty,raw,$id,$sequence,$createdAt,$updatedAt,$permissions,$databaseId,$collectionId
0,68d446e7bf3ed043310a,4972c2c4-65b8-4fb2-b97a-8bc7fca6d2c2,2025092401645717-1,-95.35,EUR,2025-09-24,None,2025-09-24,Freshto Ideal Clerkenwell,Freshto Ideal,"{""transactionId"":""2025092401645717-1"",""entryRe...",68d5c0030031e24894b2,353,2025-09-25T22:19:47.852+00:00,2025-09-25T22:19:47.852+00:00,[],68d42ac20031b27284c9,transactions_dev
1,68d446e7bf3ed043310a,4972c2c4-65b8-4fb2-b97a-8bc7fca6d2c2,2025092401645716-1,15.00,EUR,2025-09-23,None,2025-09-23,"Cab sharing, thank you!",Jennifer Houston,"{""transactionId"":""2025092401645716-1"",""entryRe...",68d5c0030037720ec6e8,354,2025-09-25T22:19:47.929+00:00,2025-09-25T22:19:47.929+00:00,[],68d42ac20031b27284c9,transactions_dev
2,68d446e7bf3ed043310a,4972c2c4-65b8-4fb2-b97a-8bc7fca6d2c2,2025092401645720-1,-95.35,EUR,2025-09-23,None,2025-09-23,Freshto Ideal Clerkenwell,Freshto Ideal,"{""transactionId"":""2025092401645720-1"",""entryRe...",68d5c003003c69f992f4,355,2025-09-25T22:19:48.011+00:00,2025-09-25T22:19:48.011+00:00,[],68d42ac20031b27284c9,transactions_dev
3,68d446e7bf3ed043310a,4972c2c4-65b8-4fb2-b97a-8bc7fca6d2c2,2025092401645715-1,-950.00,EUR,2025-09-22,None,2025-09-22,Monthly Rent Victoria Embankment 45,Liam Brown,"{""transactionId"":""2025092401645715-1"",""entryRe...",68d5c0040003663bddee,356,2025-09-25T22:19:48.096+00:00,2025-09-25T22:19:48.096+00:00,[],68d42ac20031b27284c9,transactions_dev
4,68d446e7bf3ed043310a,4972c2c4-65b8-4fb2-b97a-8bc7fca6d2c2,2025092401645719-1,15.00,EUR,2025-09-22,None,2025-09-22,"Cab sharing, thank you!",Jennifer Houston,"{""transactionId"":""2025092401645719-1"",""entryRe...",68d5c0040008dee7f0ef,357,2025-09-25T22:19:48.175+00:00,2025-09-25T22:19:48.175+00:00,[],68d42ac20031b27284c9,transactions_dev
5,68d446e7bf3ed043310a,4972c2c4-65b8-4fb2-b97a-8bc7fca6d2c2,2025092401645723-1,-95.35,EUR,2025-09-22,None,2025-09-22,Freshto Ideal Clerkenwell,Freshto Ideal,"{""transactionId"":""2025092401645723-1"",""entryRe...",68d5c004000db81258c1,358,2025-09-25T22:19:48.260+00:00,2025-09-25T22:19:48.260+00:00,[],68d42ac20031b27284c9,transactions_dev
6,68d446e7bf3ed043310a,4972c2c4-65b8-4fb2-b97a-8bc7fca6d2c2,2025092401645714-1,-35.56,EUR,2025-09-21,None,2025-09-21,Water invoice #5678,Liam Brown,"{""transactionId"":""2025092401645714-1"",""entryRe...",68d5c0040012de251db8,359,2025-09-25T22:19:48.343+00:00,2025-09-25T22:19:48.343+00:00,[],68d42ac20031b27284c9,transactions_dev
7,68d446e7bf3ed043310a,4972c2c4-65b8-4fb2-b97a-8bc7fca6d2c2,2025092401645718-1,-950.00,EUR,2025-09-21,None,2025-09-21,Monthly Rent Victoria Embankment 45,Liam Brown,"{""transactionId"":""2025092401645718-1"",""entryRe...",68d5c0040017ca56d1c3,360,2025-09-25T22:19:48.422+00:00,2025-09-25T22:19:48.422+00:00,[],68d42ac20031b27284c9,transactions_dev
8,68d446e7bf3ed043310a,4972c2c4-65b8-4fb2-b97a-8bc7fca6d2c2,2025092401645722-1,15.00,EUR,2025-09-21,None,2025-09-21,"Cab sharing, thank you!",Jennifer Houston,"{""transactionId"":""2025092401645722-1"",""entryRe...",68d5c0040021c6ebdf5d,361,2025-09-25T22:19:48.583+00:00,2025-09-25T22:19:48.583+00:00,[],68d42ac20031b27284c9,transactions_dev
9,68d446e7bf3ed043310a,4972c2c4-65b8-4fb2-b97a-8bc7fca6d2c2,2025092401645713-1,-24.56,EUR,2025-09-20,None,2025-09-20,Freshto Ideal Clerkenwell,Freshto Ideal,"{""transactionId"":""2025092401645713-1"",""entryRe...",68d5c0040026fe7bd91f,362,2025-09-25T22:19:48.668+00:00,2025-09-25T22:19:48.668+00:00,[],68d42ac20031b27284c9,transactions_dev


In [21]:
df.bookingDate.min(), df.bookingDate.max()

('2025-09-08', '2025-09-24')